In [1]:
from transformers import AutoTokenizer, AutoModel
import torch  

from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_transformers import LongContextReorder
import faiss
import os
import joblib
from langchain_community.docstore.in_memory import InMemoryDocstore
from faiss import IndexFlatL2

/home/simon/miniconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Author: Simon Zhong

In [2]:
# example
#only in test condition import
from multilinggual_dataset_processing.data_processing import EmbeddingModel
from retriever_module.retriever import aggregate_vstores,prompt_router,llm_wrapper,ChineseTemplate_0,EnglishTemplate_0,EnglishTemplate
import importlib


embedder = EmbeddingModel(model_name_or_path="/home/simon/disk1/Simon/Code/COLING/models/BAAI/bge-reranker-v2-m3")
embed_dims = embedder.encode("test").shape[1] # dim of embedder
# EmbeddingModel = importlib.import_module("data_processing").EmbeddingModel
docstore = [FAISS.load_local(folder_path="/home/simon/disk1/Simon/Code/COLING/RAG_data/FAISS_DB/", embeddings=embedder,allow_dangerous_deserialization=True)]
# 数据库整合
docstore = aggregate_vstores(docstore,embedder)


2024-08-18 16:52:12.897292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-18 16:52:12.904203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:483] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1723971132.911351 2304717 cuda_dnn.cc:8458] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1723971132.913475 2304717 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-18 16:52:12.919698: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    # ChatPromptTemplate.from_template(EnglishTemplate) | ChatOpenAI(temperature=0) | StrOutputParser() 
    ChatPromptTemplate.from_template(EnglishTemplate_0) | llm_wrapper(model_path_or_name="/home/simon/disk1/Simon/Code/LLM/models/meta-llama/Llama-2-7b-hf") | StrOutputParser() 
)

In [5]:
# generate_docs_for_retrieval.invoke("text")

In [5]:
retriever = docstore.as_retriever()
retrieval_chain = generate_docs_for_retrieval |RunnableLambda(lambda x : [x])| retriever 

# example: 
question = "whta is AI LAW"
retireved_docs = retrieval_chain.invoke({"query":question})

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Making API call to Together endpoint.


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.85s/it]
/home/simon/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/simon/miniconda3/envs/LLM/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


# For Custom HuggingFace Model 

In [7]:

llama_llm =  llm_wrapper(model_path_or_name="/home/simon/disk1/Simon/Code/LLM/models/meta-llama/Llama-2-7b-hf")
chain_rag = (
    {"query": RunnablePassthrough(),"context":retriever}
    | ChatPromptTemplate.from_template(EnglishTemplate)
    | llama_llm
    | StrOutputParser()

)

- To do 
- ADD CUSTOM llm

In [18]:
llama_llm.invoke("what is AI LAW?")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Making API call to Together endpoint.


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]


'what is AI LAW?\n Hinweis: Die Anmeldung ist nur in englischer Sprache möglich.\nThe AI LAW is a 3-day conference on Artificial Intelligence and Law. It is the first conference of its kind in Europe and will take place in Vienna, Austria, on 21-23 June 2018.\nThe AI LAW is a joint initiative of the University of Vienna, the University of Graz, the University of Salzburg,'

In [8]:
llama_llm("what is ai law")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Making API call to Together endpoint.


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.05s/it]


'what is ai law?\n nobody is going to be able to tell you what is ai law.\n\nComment: @DmitryGrigoryev I\'m not sure what you mean by "what is ai law". I\'m not sure what you mean by "what is ai law".\n\nComment: @DmitryGrigoryev I\'m not sure what you mean by "what is ai law".\n\nComment: @DmitryGrigoryev I\'m'

In [9]:
chain_rag.invoke("when is the AI LAW published") 

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Making API call to Together endpoint.


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]


"Human: \nYou are a very good English native speaker. You are great at answering questions in English. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together in English to answer the broader question.\nHere is the provided context:\n[Document(page_content='high-risk AI system. The declaration shall contain all the information required for identification of the Union harmonisation legislation to which the declaration relates. ', metadata={'source': '/home/simon/disk1/Simon/Code/COLING/Multilingual AI LAW/English.doc', 'category_depth': 0, 'file_directory': '/home/simon/disk1/Simon/Code/COLING/Multilingual AI LAW', 'filename': 'English.doc', 'last_modified': '2024-06-15T01:11:06', 'page_number': 69, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'category': 'NarrativeText'}), Document(page_content='7.\\tNotified bodies shall have proce

# For Custom OpenAI

In [6]:
from generate_module.generate import Generate,API_LLM_library
openai_35 = API_LLM_library(api_name="gpt-3.5-turbo")        

[TencentCloudSDKException] code:AuthFailure.SecretIdNotFound message:SecretId不存在，请输入正确的密钥。 requestId:d38e52dd-73c4-445b-a74e-18aefa31c540


In [7]:
openai_35.invoke("what is the AI's LAW?")

Making API call to  endpoint.


"AI's Law refers to the legal and ethical rules that govern the development and use of artificial intelligence (AI). This is relatively new legal territory, and specific legislation can vary greatly depending on location and context. However, there are some general principles that many agree should guide AI law:\n\n1. Transparency: AI systems should be transparent and explainable. People should be able to understand how the system works and how decisions are made.\n\n2. Accountability: If an AI system causes harm, there should be a clear party responsible. This could be the developer, the user, or even the AI system itself in some cases.\n\n3. Privacy: AI systems should respect the privacy of individuals. This includes not collecting or using personal data without permission.\n\n4. Fairness: AI systems should not discriminate or show bias. This means they should be designed and trained in a way that does not favor certain groups over others.\n\n5. Security: AI systems should be secure 

In [12]:
retriever = docstore.as_retriever()

In [13]:
LLM_library = Generate()
LLM_library.get_gpt_answer("what is AI LAW?","what is AI LAW?","what is AI LAW?")

request hyllm err:HTTPConnectionPool(host='hunyuanapi.woa.com', port=80): Max retries exceeded with url: /openapi/v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f191d60dcc0>, 'Connection to hunyuanapi.woa.com timed out. (connect timeout=60)'))


("HTTPConnectionPool(host='hunyuanapi.woa.com', port=80): Max retries exceeded with url: /openapi/v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7f191d60dcc0>, 'Connection to hunyuanapi.woa.com timed out. (connect timeout=60)'))",
 "AI Law refers to the area of law that deals with the legal implications of Artificial Intelligence. This includes aspects like regulation of AI and its applications, intellectual property rights of AI systems, liability issues with AI, data privacy and protection concerns associated with AI, and more. Given the widespread use and impact of AI in today's world, there is a need for specific laws and regulations to govern its use and address potential legal and ethical issues.",
 'AI Law, also known as Artificial Intelligence Law, is a branch of law that addresses the legal concerns and challenges associated with AI (Artificial Intelligence), which includes autonomous systems, machine learning, robotics, and m

In [14]:
openai_chain_rag = (
    {"query": RunnablePassthrough(),"context":retriever}
    | ChatPromptTemplate.from_template(EnglishTemplate)
    | openai_35
    | StrOutputParser()
)

In [15]:
A = openai_chain_rag.invoke("what is AI LAW")

Making API call to  endpoint.


In [16]:
A

'AI Law refers to the legal and regulatory norms, rules, and principles applicable to artificial intelligence (AI). It involves the establishment of a legal framework that governs the use, creation, distribution, and consequences of AI technologies and applications. The documents provided touch upon topics such as AI regulatory sandboxes and certifications which are part of this broader area of law. For instance, an AI regulatory sandbox is a testing environment used to experiment with AI technologies within a controlled setting under the oversight of competent authorities. It is considered vital for the promotion of AI, as indicated in one of the documents.'

# For Custom Hunyuan

In [17]:
# same as openai ,changing the api_name value to "hunyuan"

# To Do
- multilingual rag

In [17]:
import numpy as np  
  
def kl_divergence(p, q):  
    """  
    计算KL散度 D_KL(p||q)  
    :param p: 第一个概率分布，numpy数组  
    :param q: 第二个概率分布，numpy数组  
    :return: KL散度的值  
    """  
    # 确保p和q都是概率分布，即它们的和都为1  
    p = p / np.sum(p)  
    q = q / np.sum(q)  
      
    # 避免对0取对数，将0替换为一个非常小的正数（如1e-15）  
    p = np.clip(p, 1e-15, 1)  
    q = np.clip(q, 1e-15, 1)  
      
    # 计算KL散度  
    return np.sum(p * np.log(p / q))  
  
# 示例  
p = np.array([0.1, 0.2, 0.7,0.1, 0.2, 0.7,0.1, 0.2, 0.7,0.1, 0.2, 0.7])  
q = np.array([0.2, 0.2, 0.6,0.2, 0.2, 0.6,0.2, 0.2, 0.6,0.2, 0.2, 0.6])  
kl_pq = kl_divergence(p, q)  
kl_qp = kl_divergence(q, p)  
print(f"D_KL(p||q) = {kl_pq}, D_KL(q||p) = {kl_qp}")

D_KL(p||q) = 0.038590757823086505, D_KL(q||p) = 0.04613902821563383


In [16]:
def js_divergence(p, q):  
    """  
    计算JS散度  
    :param p: 第一个概率分布，numpy数组  
    :param q: 第二个概率分布，numpy数组  
    :return: JS散度的值  
    """  
    # 确保p和q都是概率分布  
    p = p / np.sum(p)  
    q = q / np.sum(q)  
      
    # 计算混合分布M  
    m = 0.5 * (p + q)  
      
    # 计算KL散度 D_KL(p||m) 和 D_KL(q||m)  
    kl_pm = np.sum(p * np.log(p / m))  
    kl_qm = np.sum(q * np.log(q / m))  
      
    # 计算JS散度  
    return 0.5 * (kl_pm + kl_qm)  
  
# 示例  
p = np.array([0.1, 0.2, 0.7])  
q = np.array([0.0008, 0.9, 1.6])  
js_pq = js_divergence(p, q)  
print(f"JSD(p, q) = {js_pq}")

JSD(p, q) = 0.04594059884701647


In [20]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('/home/simon/disk1/Simon/Code/LLM/models/intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('/home/simon/disk1/Simon/Code/LLM/models/intfloat/multilingual-e5-base')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


[[90.79102325439453, 71.17613983154297], [72.53498840332031, 90.14605712890625]]
